# Visualize the Keywords using LDA Algorithm

In [1]:
import pandas as pd
import gensim
import jieba

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.manifold import TSNE

from wordcloud import WordCloud

from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()


### BEGIN settings of pyLDAvis ### 
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
### END settings of pyLDAvis ###

Loading BokehJS ...

# Prepare the Dataset

In [2]:
def read_json_dataset(filename):
    """
    Read JSON dataset and return the job requirements
    
    Returns:
        job requirements
    """
    dataset = pd.read_json(filename)
    
    return dataset['requirement'] + ' ' + dataset['requirement_others']


def tokenizer(doc):
    return jieba.cut(doc, cut_all=False)


def get_chinese_stopwords():
    """
    Get Chinese stopwords
    """
    with open('./中文停用词表.txt') as f:
        lines = f.readlines()
        stopwords_chinese = [w.strip() for w in lines]
        
    return stopwords_chinese

    
def preprocess(raw_docs):
    """
    Normalize, tokenize, remove stopwords, use custom dictionary
    
    Args:
        raw_docs (list(str)):
        
    Returns:
        docs (list(list(str))): list of tokens in a document
    """
    
    docs = []
    
    # define customized stopwords
    stopwords_custom = ['•', '與', '★', '●', '（', '’', '－', '✦', '◆', '◼', '✪', 
                        '※', '⁺', '', '', '·', '‧', '・', '）', '○', '】', '【', '✓', '']
    
    stopwords_chinese = get_chinese_stopwords()
    
    for d in tqdm(raw_docs):
        # Normalize English words
        d = d.lower()
        
        tokens = []
        
        for t in tokenizer(d):  
            # Strip English punctuations
            t = gensim.parsing.preprocessing.strip_punctuation(t)
            
            # Remove numbers
            t = gensim.parsing.preprocessing.strip_numeric(t)
            
            t = t.strip()
            
            if t is '':
                continue
                
            if t not in stopwords_custom:
                if t not in stopwords_chinese:
                    if t not in gensim.parsing.preprocessing.STOPWORDS:
                        tokens.append(t)
                    
        docs.append(tokens)
        
    return docs


def get_dictionary(docs):
    return gensim.corpora.Dictionary(docs)


def get_corpus_bow(docs, dictionary):
    """
    Get corpus with format of BOW
    
    Args:
        docs (list(list(str))): list of list of string token
        dictionary (gensim.corpora.Dictionary): dictionary
        
    Returns:
        bows (list(list(tuple(token_id, num_tokens))))
    """
    bows = [dictionary.doc2bow(d) for d in docs]
    
    return bows


def get_corpus_tfidf(bows):
    """
    Get corpus based on TF-IDF and TF-IDF model
    
    Args:
        bows (list(list(tuple(token_id, num_tokens)))): BOW
        
    Returns:
        corpus_tfidf (gensim.interfaces.TransformedCorpus): corpus class
        tfidf_model: 
    """
    tfidf_model = gensim.models.TfidfModel(bows)
    corpus_tfidf = tfidf_model[bows]
    
    return corpus_tfidf, tfidf_model


In [3]:
filename_dataset = '../crawler/employment_website_104/dataset/jobs_104.json'
no_below = 6
no_above = 0.1
passes = 20
iterations = 400

In [4]:
raw_docs = read_json_dataset(filename_dataset)

# Normalize, tokenize, remove stopwords, use custom dictionary
docs = preprocess(raw_docs)

# Create dictionary
dictionary = get_dictionary(docs)
print('Size of dictionary before filtering out extreme words: {}'.format(len(dictionary)))

# Remove common/rare words
dictionary.filter_extremes(no_below=no_below, no_above=no_above)
print('Size of dictionary after filtering out extreme words: {}'.format(len(dictionary)))

# generate BOW corpus
corpus_bow = get_corpus_bow(docs, dictionary)

# generate TF-IDF corpus using BOW
corpus_tfidf, tfidf_model = get_corpus_tfidf(corpus_bow)

  0%|          | 0/996 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.965 seconds.
Prefix dict has been built succesfully.
100%|██████████| 996/996 [00:07<00:00, 133.65it/s]


Size of dictionary before filtering out extreme words: 10764
Size of dictionary after filtering out extreme words: 2444


In [5]:
type(corpus_tfidf)

gensim.interfaces.TransformedCorpus

# Choose Number of Topics of BOW using pyLDAvis

In [27]:
num_topics = 4

# train BOW model
lda_model_bow = gensim.models.LdaModel(corpus_bow,
                                       num_topics=num_topics,
                                       id2word=dictionary,
                                       passes=passes,
                                      iterations=iterations,
                                      alpha='auto', eta='auto')


In [28]:
pyLDAvis.gensim.prepare(lda_model_bow, corpus_bow, dictionary)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.138829 -0.267105       1        1  49.595943
2      0.130493 -0.005464       2        1  28.599239
0     -0.359138 -0.029656       3        1  10.961351
1      0.089815  0.302225       4        1  10.843470, topic_info=     Category        Freq  Term       Total  loglift  logprob
211   Default  317.000000     舖  317.000000  30.0000  30.0000
205   Default  269.000000    經營  269.000000  29.0000  29.0000
155   Default  182.000000    主任  182.000000  28.0000  28.0000
172   Default  176.000000    培訓  176.000000  27.0000  27.0000
171   Default  156.000000    培育  156.000000  26.0000  26.0000
166   Default  137.000000    分店  137.000000  25.0000  25.0000
299   Default  218.000000    環境  218.000000  24.0000  24.0000
1257  Default  124.000000    獎金  124.000000  23.0000  23.0000
212   Default  116.000000    表現  116.000000  22.0000  22.0000
169   Default  116.000000     區  116.000000  21.0000  21.0000
204   Default  118.000000     約  118.000000  20.0000  20.0000
1044  Default  113.000000    完善  113.000000  19.0000  19.0000
1264  Default  136.000000    門市  136.000000  18.0000  18.0000
230   Default  107.000000     顧  107.000000  17.0000  17.0000
174   Default  108.000000    客服  108.000000  16.0000  16.0000
77    Default  193.000000    數據  193.000000  15.0000  15.0000
201   Default  150.000000    發展  150.000000  14.0000  14.0000
100   Default  159.000000     請  159.000000  13.0000  13.0000
1240  Default   99.000000    制度   99.000000  12.0000  12.0000
1251  Default  106.000000    教育  106.000000  11.0000  11.0000
1774  Default  101.000000    福利  101.000000  10.0000  10.0000
154   Default   91.000000    三班   91.000000   9.0000   9.0000
1742  Default   94.000000    年度   94.000000   8.0000   8.0000
72    Default  104.000000     想  104.000000   7.0000   7.0000
1763  Default   88.000000     次   88.000000   6.0000   6.0000
214   Default   87.000000    課堂   87.000000   5.0000   5.0000
157   Default   87.000000  交通工具   87.000000   4.0000   4.0000
228   Default   87.000000   須自備   87.000000   3.0000   3.0000
1807  Default   85.000000     餐   85.000000   2.0000   2.0000
206   Default   91.000000    經理   91.000000   1.0000   1.0000
...       ...         ...   ...         ...      ...      ...
1257   Topic4  121.016800    獎金  124.013290   2.1971  -4.1039
1812   Topic4   32.180000    駕照   32.725056   2.2048  -5.4285
1242   Topic4   60.606346    升遷   62.409859   2.1923  -4.7954
1742   Topic4   90.486229    年度   94.048828   2.1830  -4.3946
1781   Topic4   58.814636    績效   60.641754   2.1910  -4.8255
1715   Topic4   56.050842    一展   57.815613   2.1906  -4.8736
1263   Topic4   33.969185     金   34.674362   2.2011  -5.3744
1780   Topic4   30.346964    結帳   30.884771   2.2040  -5.4871
1432   Topic4   30.346697    工讀   30.898569   2.2036  -5.4872
1732   Topic4   30.343628    外向   30.904615   2.2033  -5.4873
1765   Topic4   30.344063    活潑   30.905876   2.2033  -5.4872
1774   Topic4   90.489555    福利  101.499649   2.1068  -4.3946
1787   Topic4   58.844723    職能   63.665104   2.1429  -4.8249
1264   Topic4  116.107056    門市  136.495239   2.0598  -4.1453
1251   Topic4   92.905952    教育  106.826973   2.0820  -4.3683
1717   Topic4   58.861553    介紹   64.686478   2.1272  -4.8247
1261   Topic4   35.389404    補助   36.856010   2.1810  -5.3334
72     Topic4   86.446075     想  104.257317   2.0343  -4.4403
1734   Topic4   43.813156    外派   48.024479   2.1298  -5.1199
1245   Topic4   36.525654    員工   38.618011   2.1659  -5.3018
1740   Topic4   32.059967    對餐   32.883453   2.1962  -5.4322
299    Topic4  122.836205    環境  218.950165   1.6436  -4.0890
67     Topic4   59.597164    履歷   79.116264   1.9383  -4.8122
1750   Topic4   57.007294    投遞   75.189865   1.9448  -4.8567
1472   Topic4   59.276249    安全   84.724960   1.8644  -4.8176
100    Topic4   86.772125     請  159.479660   1.6130  -4.4366
226    Topic4   60.550396     長  112.

# Select Num of Topics of TF-IDF using pyLDAvis

In [23]:
num_topics = 4
# train TF-IDF model
lda_model_tfidf = gensim.models.LdaModel(corpus_tfidf,
                                        num_topics=num_topics,
                                        id2word=dictionary,
                                        passes=passes,
                                        iterations=iterations,
                                      alpha='auto', eta='auto')

In [24]:
pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.052417 -0.024303       1        1  44.123207
3     -0.060544 -0.018663       2        1  39.927670
2      0.003568  0.052373       3        1  12.181873
0      0.004559 -0.009407       4        1   3.767256, topic_info=     Category       Freq           Term      Total  loglift  logprob
211   Default  12.000000              舖  12.000000  30.0000  30.0000
205   Default  10.000000             經營  10.000000  29.0000  29.0000
155   Default   7.000000             主任   7.000000  28.0000  28.0000
172   Default   8.000000             培訓   8.000000  27.0000  27.0000
171   Default   6.000000             培育   6.000000  26.0000  26.0000
166   Default   5.000000             分店   5.000000  25.0000  25.0000
169   Default   5.000000              區   5.000000  24.0000  24.0000
204   Default   5.000000              約   5.000000  23.0000  23.0000
212   Default   5.000000             表現   5.000000  22.0000  22.0000
201   Default   8.000000             發展   8.000000  21.0000  21.0000
174   Default   4.000000             客服   4.000000  20.0000  20.0000
230   Default   4.000000              顧   4.000000  19.0000  19.0000
8     Default   8.000000       customer   8.000000  18.0000  18.0000
424   Default   6.000000      equipment   6.000000  17.0000  17.0000
154   Default   4.000000             三班   4.000000  16.0000  16.0000
299   Default  10.000000             環境  10.000000  15.0000  15.0000
157   Default   3.000000           交通工具   3.000000  14.0000  14.0000
214   Default   3.000000             課堂   3.000000  13.0000  13.0000
206   Default   4.000000             經理   4.000000  12.0000  12.0000
228   Default   3.000000            須自備   3.000000  11.0000  11.0000
0     Default  10.000000      algorithm  10.000000  10.0000  10.0000
555   Default   7.000000          skill   7.000000   9.0000   9.0000
217   Default   5.000000             資源   5.000000   8.0000   8.0000
177   Default   3.000000            店經理   3.000000   7.0000   7.0000
436   Default   5.000000  manufacturing   5.000000   6.0000   6.0000
187   Default   4.000000              月   4.000000   5.0000   5.0000
189   Default   4.000000            有興趣   4.000000   4.0000   4.0000
162   Default   3.000000           便利商店   3.000000   3.0000   3.0000
237   Default  12.000000             影像  12.000000   2.0000   2.0000
173   Default   5.000000             大量   5.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
661    Topic4   0.299746           digi   3.843083   0.7277  -6.6588
670    Topic4   0.317096            經濟部   4.580518   0.6085  -6.6025
555    Topic4   0.436819          skill   7.552977   0.4286  -6.2822
557    Topic4   0.283131       teamwork   4.032710   0.6225  -6.7158
664    Topic4   0.341798         talent   5.446679   0.5103  -6.5275
671    Topic4   0.300712             計畫   4.592737   0.5527  -6.6556
714    Topic4   0.352107           able   6.193197   0.4116  -6.4978
0      Topic4   0.465173      algorithm  10.973338   0.1180  -6.2193
350    Topic4   0.325295      excellent   5.651321   0.4239  -6.5770
438    Topic4   0.305406      operation   5.442047   0.3986  -6.6401
134    Topic4   0.361125         neural   7.906115   0.1927  -6.4725
1016   Topic4   0.383639        service   9.096237   0.1129  -6.4120
673    Topic4   0.347773             面試   7.327098   0.2310  -6.5102
4      Topic4   0.295141       attitude   5.116734   0.4260  -6.6743
390    Topic4   0.340631          sales   7.242585   0.2219  -6.5309
443    Topic4   0.362155        project   8.882166   0.0791  -6.4697
141    Topic4   0.341913       problems   7.761858   0.1564  -6.5272
665    Topic4   0.330604             優先   7.164099   0.2029  -6.5608
19     Topic4   0.413693          image  13.106380  -0.1769  -6.3366
735    Topic4   0.360255        network   9.321217   0.0256  -6.4749
9      Topic4   0.319517      customers   7.1